In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect, Table, Column, MetaData, Integer, String, Float
import urllib
from fuzzywuzzy import process

# Create connection string for pyodbc
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=Server-Name;'
    'DATABASE=DB_Name;'
    'Trusted_Connection=yes;'
)

# URL encode the connection string
conn_str = urllib.parse.quote_plus(conn_str)

# Create a SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}')

# Read Excel file
excel_data = pd.read_excel('C:/Users/user.name/Desktop/Excel_Name.xlsx')
print("Excel data columns:", excel_data.columns)

# Get SQL table column names
inspector = inspect(engine)
table_name = 'Table_Name'
if table_name in inspector.get_table_names():
    sql_columns = [column['name'] for column in inspector.get_columns(table_name)]
    print("SQL table columns:", sql_columns)
else:
    # Create table if it doesn't exist
    metadata = MetaData()
    columns = []
    for col in excel_data.columns:
        dtype = excel_data[col].dtype
        if dtype == 'int64':
            col_type = Integer  
        elif dtype == 'float64':
            col_type = Float
        else:
            col_type = String
        columns.append(Column(col, col_type))
    table = Table(table_name, metadata, *columns)
    metadata.create_all(engine)
    sql_columns = excel_data.columns.tolist()
    print(f"Created table '{table_name}' with columns:", sql_columns)

# Function to match Excel columns to SQL columns
def match_columns(excel_cols, sql_cols):
    column_mapping = {}
    for excel_col in excel_cols:
        match, score = process.extractOne(excel_col, sql_cols)
        if score > 80:  # You can adjust the threshold as needed
            column_mapping[excel_col] = match
    return column_mapping

# Match columns using fuzzy matching
column_mapping = match_columns(excel_data.columns, sql_columns)
print("Column mapping:", column_mapping)

# Apply the column mappings
excel_data = excel_data.rename(columns=column_mapping)
print("Renamed Excel data columns:", excel_data.columns)

# Find common columns between Excel and SQL table
common_columns = list(set(excel_data.columns).intersection(sql_columns))
print("Common columns:", common_columns)

# Ensure DataFrame only includes common columns, ordered as per SQL table
excel_data_filtered = excel_data[common_columns]
print("Filtered DataFrame columns:", excel_data_filtered.columns)

# Re-order columns to match SQL table order, ignoring columns not in common_columns
ordered_columns = [col for col in sql_columns if col in common_columns]
excel_data_filtered = excel_data_filtered[ordered_columns]
print("Reordered DataFrame columns:", excel_data_filtered.columns)

# Write DataFrame to SQL Server
excel_data_filtered.to_sql(table_name, engine, if_exists='replace', index=False)
print("Data inserted successfully.")
